#### Indian sign laguages (CNN + RNN)

In [1]:
''' keras video genrator for data augmentation of the video'''
!pip install keras-video-generators
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

' keras video genrator for data augmentation of the video'

In [2]:
%cd /content/drive/My Drive/PROJECT_DATA

/content/drive/My Drive/PROJECT_DATA


In [3]:
'''importing libraries'''
import os
import sys
import glob
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from keras_video import VideoFrameGenerator

Using TensorFlow backend.


In [4]:
from keras_video import VideoFrameGenerator
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('hands_only_zoomed/*')]
classes.sort()
# some global params
SIZE = (112, 112)
CHANNELS = 3
NBFRAME = 8
BS = 8
# pattern to get videos and classes
glob_pattern='hands_only_zoomed/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.7)
    # horizontal_flip=True,
    # rotation_range=8,
    # width_shift_range=.2,
    # height_shift/_range=.2)
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.3, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

class friday, validation count: 61, train count: 143
class monday, validation count: 69, train count: 161
class saturday, validation count: 61, train count: 145
class sunday, validation count: 69, train count: 162
class thursday, validation count: 61, train count: 143
class tuesday, validation count: 79, train count: 185
class wednesday, validation count: 63, train count: 148
Total data: 7 classes for 1087 files for train


In [5]:
valid = train.get_validation_generator()

Total data: 7 classes for 463 files for validation


In [0]:
# import keras_video.utils
# keras_video.utils.show_sample(train)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical

In [9]:
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(8, 112,112,3)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_3 (ConvLSTM2D)  (None, 112, 110, 64)      51712     
_________________________________________________________________
dropout_3 (Dropout)          (None, 112, 110, 64)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 788480)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               78848100  
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 707       
Total params: 78,900,519
Trainable params: 78,900,519
Non-trainable params: 0
_________________________________________________________________


In [0]:
from sklearn.utils import class_weight
def get_weight(y):
    class_weight_current =  class_weight.compute_class_weight('balanced', np.unique(y), y)
    return class_weight_current

In [0]:
# to check the class balancing requred ??
class_weight = get_weight(train.classes)
class_weight

In [10]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),metrics=['accuracy'])

In [0]:
# callbacks used in the model training part.....
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
filepath="/content/drive/My Drive/Models/final.hdf5" # to save the best weights tot eh path
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True, verbose=1)
reduce_on_plateau = ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.1, patience=4, verbose=1)
early_stopping = EarlyStopping( monitor='val_loss',patience=10,verbose=1,restore_best_weights=True)
callbacks_list = [checkpoint, reduce_on_plateau, early_stopping]

In [12]:
# model training
history = model.fit_generator(train, epochs=20,
                              callbacks=callbacks_list, verbose = 1,
                              validation_data = valid,
                              workers=-1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





113/135 [========================>.....] - ETA: 1:21 - loss: 1.8001 - acc: 0.3086

Frame count is not OK for video hands_only_zoomed/sunday/SUNDAY_2.mp4, 5 total, 5 extracted


124/135 [==========================>...] - ETA: 40s - loss: 1.7866 - acc: 0.3180

Frame count is not OK for video hands_only_zoomed/sunday/SUNDAY_150.mp4, 6 total, 6 extracted


134/135 [============================>.] - ETA: 3s - loss: 1.7791 - acc: 0.3190

...

 58/135 [===========>..................] - ETA: 25s - loss: 0.4455 - acc: 0.8639

Frame count is not OK for video hands_only_zoomed/sunday/SUNDAY_150.mp4, 6 total, 6 extracted


134/135 [============================>.] - ETA: 0s - loss: 0.4100 - acc: 0.8805

Frame count is not OK for video hands_only_zoomed/thursday/THURSDAY157.mp4, 6 total, 6 extracted
Frame count is not OK for video hands_only_zoomed/thursday/THURSDAY32.mp4, 7 total, 7 extracted


135/135 [==============================] - 49s 362ms/step - loss: 0.4082 - acc: 0.8813 - val_loss: 0.6193 - val_acc: 0.8040

Epoch 00020: val_loss did not improve from 0.53474
